<a href="https://colab.research.google.com/github/MarwanAhmed20/NLP-Similarity-texts-wikipedia_api/blob/main/Similarity_texts_using_wikipedia_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Python Libraries

In [ ]:
import wikipediaapi
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

import gensim.models.keyedvectors as word2vec

# Load wikipediaapi dataset with api

In [ ]:
# use words to load data relatied
doc_search = ["Car","bus","train","ship","mobile","laptop","personal computer","xbox","pop songs","rap","Hip-Hop","folk songs","sandbox games","simulation games","Action-adventure games","Multiplayer online battle arena","football","basketball","volleyball","handball"]
wiki = wikipediaapi.Wikipedia("en")

data = []
for doc in doc_search:
    page = wiki.page(doc)
    text = ""
    for i in range(len(page.sections)):
        text += page.sections[i].text
    data.append(text)


In [ ]:
print(data[8])

David Hatch and Stephen Millward define pop music as "a body of music which is distinguishable from popular, jazz, and folk music".
According to Pete Seeger, pop music is "professional music which draws upon both folk music and fine arts music".
David Boyle, a music researcher, states pop music as any type of music that a person has been exposed to by the mass media. Most individuals think that pop music is just the singles charts and not the sum of all chart music. The music charts contain songs from a variety of sources, including classical, jazz, rock, and novelty songs. As a genre, pop music is seen to exist and develop separately. Therefore, the term "pop music" may be used to describe a distinct genre, designed to appeal to all, often characterized as "instant singles-based music aimed at teenagers" in contrast to rock music as "album-based music for adults".Pop music continuously evolves along with the term's definition. According to music writer Bill Lamb, popular music is defi

# preprocessing dataset

In [ ]:
def preprocessing (text):
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“'~–'''
    for x in text.lower():
        if x in punctuations :
            text = text.replace(x, "")
        elif x == '\n':
            text = text.replace(x, " ")
    text = text.lower()
    text = re.sub(r'\s+[a-zA-Z]\s+','', text)
    text = re.sub('[0-9]+', '', text)
    text = text.split(' ')
    text = [word for word in text if word not in en_stop and len(word)!= 1]
    text= [lemmatizer.lemmatize(word) for word in text]
    text = [x for x in text if x!='']
    text = set(text)
    return text

In [ ]:
lemmatizer = WordNetLemmatizer()
en_stop = set(stopwords.words('english'))
docs = []
for i in range(len(data)):
    docs.append(preprocessing(data[i]))

# Apply word2vec model

In [ ]:
model = word2vec.KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

In [ ]:
words_vector = []
for doc in docs:
    doc_vector = []
    for word in doc:
        if word in model.index_to_key:
            doc_vector.append(model[word])
    words_vector.append(doc_vector)

In [ ]:
print(words_vector[0][0])

[-2.074e-01 -6.970e-02  3.100e-02 -8.680e-02 -8.700e-02 -6.200e-03
 -1.001e-01 -1.007e-01 -2.600e-03 -1.009e-01  2.430e-02  4.430e-02
 -2.420e-02 -5.120e-02  1.030e-02 -1.290e-02  8.420e-02  3.690e-02
  2.716e-01  6.800e-03  3.330e-02  1.147e-01 -1.835e-01  2.570e-02
 -9.380e-02 -2.590e-02  1.340e-02 -4.710e-02  8.790e-02  8.080e-02
  9.550e-02  4.520e-02  7.110e-02 -1.361e-01  9.960e-02 -3.040e-02
  1.630e-02  4.110e-02 -2.210e-02 -4.450e-02 -4.260e-02  0.000e+00
 -4.180e-02  0.000e+00 -6.890e-02  1.550e-02  7.080e-02 -8.010e-02
 -9.040e-02 -1.067e-01 -1.200e-02  8.900e-02 -6.403e-01  2.470e-02
 -1.137e-01 -2.000e-02 -1.000e-04 -5.530e-02 -1.821e-01 -1.198e-01
  1.012e-01  7.340e-02  9.240e-02  3.740e-02 -1.571e-01  1.565e-01
 -1.800e-03  6.810e-02 -9.920e-02 -2.470e-02  4.890e-02  8.720e-02
  6.500e-02 -6.500e-03 -8.330e-02  8.210e-02  6.170e-02 -1.500e-02
 -5.530e-02  1.768e-01  5.000e-03  3.590e-02 -9.310e-02 -3.087e-01
  1.048e-01 -3.460e-02  2.530e-02  3.130e-02 -3.905e-01  3.180

In [ ]:
import numpy as np
doc_embedding = []
for doc in words_vector:
    sum_embedding = 0
    for word in doc:
        sum_embedding += word
    doc_embedding.append(sum_embedding/len(doc))

In [ ]:
np.save("doc_embedding.npy",doc_embedding)

# Get Similarity

In [ ]:
def get_embedding(sentence):
    text = preprocessing(sentence)
    embedding_vector = []
    for word in text:
        if word in model.index_to_key:
            embedding_vector.append(model[word])
    return embedding_vector


In [ ]:
def get_embedding_avg(vector):
    sum_embedding = 0
    for word in vector:
        sum_embedding+=word
    avg_test_vector = sum_embedding/len(test_vector)
    return avg_test_vector

In [ ]:
def similarity_result(emedding_vector):
    similarity_values = []
    for doc in doc_embedding:
        similarity_values.append(distance.cosine(doc,emedding_vector))

    sorted_search = [x for _,x in sorted(zip(similarity_values,doc_search))]

    return sorted_search

# Show results

In [ ]:
sentence = "The car industry is also facing increasing competition from the public transport sector"

In [ ]:
embedding_test_vector=get_embedding(sentence)
embedding_avg_vector=get_embedding_avg(embedding_test_vector)

In [ ]:
results = similarity_result(embedding_avg_vector)

In [ ]:
print(results)

['bus', 'train', 'Car', 'ship', 'volleyball', 'Multiplayer online battle arena', 'personal computer', 'laptop', 'mobile', 'football', 'xbox', 'simulation games', 'pop songs', 'basketball', 'Hip-Hop', 'sandbox games', 'Action-adventure games', 'rap', 'handball', 'folk songs']


In [ ]:
sentence = "The personal computer was made possible by major advances in semiconductor technology"


In [ ]:
embedding_test_vector=get_embedding(sentence)
embedding_avg_vector=get_embedding_avg(embedding_test_vector)

In [ ]:
results = similarity_result(embedding_avg_vector)


In [ ]:
print(results)

['personal computer', 'laptop', 'mobile', 'xbox', 'Multiplayer online battle arena', 'simulation games', 'Car', 'train', 'bus', 'Action-adventure games', 'sandbox games', 'pop songs', 'volleyball', 'ship', 'Hip-Hop', 'basketball', 'football', 'rap', 'handball', 'folk songs']


In [ ]:
sentence = "pop music is professional music which draws upon both folk music and fine arts music"

In [ ]:
embedding_test_vector=get_embedding(sentence)
embedding_avg_vector=get_embedding_avg(embedding_test_vector)

In [ ]:
results = similarity_result(embedding_avg_vector)

In [ ]:
print(results)

['pop songs', 'folk songs', 'Hip-Hop', 'rap', 'simulation games', 'volleyball', 'Action-adventure games', 'basketball', 'football', 'Multiplayer online battle arena', 'personal computer', 'bus', 'handball', 'Car', 'train', 'laptop', 'mobile', 'sandbox games', 'xbox', 'ship']
